<a href="https://colab.research.google.com/github/micaelCZ/Paper_Repositorio/blob/main/A%20CNN%2BENC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
datapath = 'https://raw.githubusercontent.com/hfcatucuamba/proyectoInternetworking/main/Escenario1.csv'
dataframe = pd.read_csv(datapath,low_memory=False, sep=';')
#Visualización de los 5 primeros datos
dataframe.head()


,Source IP,Source Port,Destination IP,Destination Port,Protocol,Flow Duration,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,...,Bwd IAT Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,label
0,10.0.2.15,53913,216.58.208.46,80,6,435,0,45.977.011.494.253,435,0,...,0,0,0,0,0,0,0,0,0,nonTOR
1,10.0.2.15,53913,216.58.208.46,80,6,259,0,77.220.077.220.077,259,0,...,0,0,0,0,0,0,0,0,0,nonTOR
2,10.0.2.15,53913,216.58.208.46,80,6,891,0,22.446.689.113.356,891,0,...,0,0,0,0,0,0,0,0,0,nonTOR
3,10.0.2.15,53913,216.58.208.46,80,6,1074,0,18.621.973.929.237,1074,0,...,0,0,0,0,0,0,0,0,0,nonTOR
4,10.0.2.15,53913,216.58.208.46,80,6,315,0,63.492.063.492.064,315,0,...,0,0,0,0,0,0,0,0,0,nonTOR


In [45]:
# Normalización de los datos
def dfNormalize(df):
    for feature_name in df.columns:
        df.loc[:,feature_name]= pd.to_numeric(df.loc[:,feature_name], errors='coerce').fillna(0)
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()   
        if (max_value - min_value) > 0:
            df.loc[:,feature_name] = (df.loc[:,feature_name] - min_value) / (max_value - min_value)
        else:
            df.loc[:,feature_name] = (df.loc[:,feature_name]- min_value)    
    return df


In [46]:
#Muestra el tamaño del dataframe (número de filas y columnas)
print (dataframe.shape)
dataframe = dataframe.reindex(np.random.permutation(dataframe.index)).copy()
print(dataframe.describe())
print (list(dataframe))

(7000, 29)
        Source Port   Destination Port   Protocol   Flow Duration  \
count   7000.000000        7000.000000     7000.0    7.000000e+03   
mean   42153.076571        7647.390714        6.0    4.002204e+06   
std    17081.978193       17168.713987        0.0    4.450485e+06   
min       80.000000          21.000000        6.0    1.700000e+01   
25%    38894.000000         443.000000        6.0    2.013598e+05   
50%    45337.000000         443.000000        6.0    4.409195e+05   
75%    54698.000000         443.000000        6.0    9.829612e+06   
max    61156.000000       65514.000000        6.0    1.000000e+07   

        Flow IAT Max   Flow IAT Min   Fwd IAT Max   Fwd IAT Min   Bwd IAT Max  \
count   7.000000e+03   7.000000e+03  7.000000e+03  7.000000e+03  7.000000e+03   
mean    1.148187e+06   1.023332e+05  1.079414e+06  1.035305e+05  8.884348e+05   
std     2.113319e+06   4.321051e+05  2.135818e+06  6.952060e+05  1.933856e+06   
min     1.700000e+01  -4.340000e+02  0.0000

In [47]:
#Obtiene las claves del dataframe (los nombres de las columnas)
keys = dataframe.keys()
data_to_process = dataframe[keys[4:len(keys) - 1]].copy()
x_normalised = dfNormalize(data_to_process)
print(x_normalised.describe())

        Protocol   Flow Duration   Flow Bytes/s   Flow Packets/s  \
count     7000.0     7000.000000    7000.000000      7000.000000   
mean         0.0        0.400219       0.000143         0.000167   
std          0.0        0.445049       0.011952         0.012051   
min          0.0        0.000000       0.000000         0.000000   
25%          0.0        0.020134       0.000000         0.000000   
50%          0.0        0.044090       0.000000         0.000000   
75%          0.0        0.982961       0.000000         0.000000   
max          0.0        1.000000       1.000000         1.000000   

        Flow IAT Mean   Flow IAT Std   Flow IAT Max   Flow IAT Min  \
count     7000.000000         7000.0    7000.000000    7000.000000   
mean         0.017407            0.0       0.114853       0.010376   
std          0.055836            0.0       0.211398       0.043627   
min          0.000000            0.0       0.000000       0.000000   
25%          0.000000            0.0 

<ipython-input-45-99d760175aeb>:4: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:,feature_name]= pd.to_numeric(df.loc[:,feature_name], errors='coerce').fillna(0)


In [48]:
#Cambiar las etiquetas de los datos de ‘nonTOR’ y ‘TOR’ a 1 y 0 respectivamente.
change_labels = lambda x: 1 if x == 'nonTOR' else 0


In [49]:
#crea una nueva variable llamada y_normalised que contiene los valores de la columna 
#‘label’ del dataframe original después de aplicar la función change_labels.
y_normalised = dataframe['label'].apply(change_labels)


In [50]:
# Dividir datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(x_normalised, y_normalised, test_size=0.3, random_state=42)


In [77]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from keras.layers import Reshape, UpSampling1D
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.callbacks import EarlyStopping

# Encoder
def create_model(dropout_rate=0.1):
    model = Sequential([
        Conv1D(64, 3, input_shape=(X_train.shape[1], 1), padding="same", activation="relu", kernel_constraint=MaxNorm(3)),
        MaxPooling1D(2),
        Dropout(dropout_rate),
        Conv1D(32, 3, padding="same", activation="relu", kernel_constraint=MaxNorm(3)),
        MaxPooling1D(2),
        Dropout(dropout_rate),
        Conv1D(16, 3, padding="same", activation="relu", kernel_constraint=MaxNorm(3)),
        MaxPooling1D(2),
        Dropout(dropout_rate),
        Flatten(),
        Dense(1, activation="sigmoid"),
    ])

    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model



In [75]:
!pip install scikeras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [79]:
# Definir los hiperparámetros y sus valores a probar
param_grid = {
    'batch_size': [16, 32],
    'epochs': [10, 20],
    'dropout_rate': [0.1, 0.2, 0.3],
}

# Crear el modelo de Keras con la función create_model y pasarla como estimador en GridSearchCV
model = KerasClassifier(build_fn=create_model, dropout_rate=0.1) # Establecer el valor predeterminado de dropout_rate
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(X_train, y_train)

/usr/local/lib/python3.9/dist-packages/scikeras/wrappers.py:301: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(


Epoch 1/10
205/205 [==============================] - 3s 5ms/step - loss: 0.4097 - accuracy: 0.8619
Epoch 2/10
205/205 [==============================] - 1s 5ms/step - loss: 0.2193 - accuracy: 0.9155
Epoch 3/10
205/205 [==============================] - 1s 5ms/step - loss: 0.1612 - accuracy: 0.9388
Epoch 4/10
205/205 [==============================] - 1s 5ms/step - loss: 0.1126 - accuracy: 0.9657
Epoch 5/10
205/205 [==============================] - 1s 5ms/step - loss: 0.0929 - accuracy: 0.9752
Epoch 6/10
205/205 [==============================] - 1s 5ms/step - loss: 0.0765 - accuracy: 0.9798
Epoch 7/10
205/205 [==============================] - 1s 7ms/step - loss: 0.0741 - accuracy: 0.9786
Epoch 8/10
205/205 [==============================] - 1s 7ms/step - loss: 0.0604 - accuracy: 0.9835
Epoch 9/10
205/205 [==============================] - 1s 5ms/step - loss: 0.0698 - accuracy: 0.9816
Epoch 10/10
103/103 [==============================] - 0s 2ms/step
Epoch 1/10


/usr/local/lib/python3.9/dist-packages/scikeras/wrappers.py:301: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(


205/205 [==============================] - 2s 5ms/step - loss: 0.3479 - accuracy: 0.8757
Epoch 2/10
205/205 [==============================] - 1s 5ms/step - loss: 0.1787 - accuracy: 0.9385
Epoch 3/10
205/205 [==============================] - 1s 5ms/step - loss: 0.1403 - accuracy: 0.9608
Epoch 4/10
205/205 [==============================] - 1s 5ms/step - loss: 0.1090 - accuracy: 0.9731
Epoch 5/10
205/205 [==============================] - 1s 5ms/step - loss: 0.0952 - accuracy: 0.9783
Epoch 6/10
205/205 [==============================] - 1s 5ms/step - loss: 0.1012 - accuracy: 0.9767
Epoch 7/10
205/205 [==============================] - 1s 5ms/step - loss: 0.0830 - accuracy: 0.9795
Epoch 8/10
205/205 [==============================] - 1s 5ms/step - loss: 0.0796 - accuracy: 0.9795
Epoch 9/10
205/205 [==============================] - 1s 6ms/step - loss: 0.0707 - accuracy: 0.9813
Epoch 10/10
103/103 [==============================] - 0s 3ms/step


/usr/local/lib/python3.9/dist-packages/scikeras/wrappers.py:301: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(


...

103/103 [==============================] - 2s 7ms/step - loss: 0.5060 - accuracy: 0.8473
Epoch 2/20
103/103 [==============================] - 1s 10ms/step - loss: 0.2758 - accuracy: 0.9036
Epoch 3/20
103/103 [==============================] - 1s 9ms/step - loss: 0.2214 - accuracy: 0.9186
Epoch 4/20
103/103 [==============================] - 1s 9ms/step - loss: 0.1892 - accuracy: 0.9305
Epoch 5/20
103/103 [==============================] - 1s 8ms/step - loss: 0.1636 - accuracy: 0.9397
Epoch 6/20
103/103 [==============================] - 1s 7ms/step - loss: 0.1454 - accuracy: 0.9495
Epoch 7/20
103/103 [==============================] - 1s 7ms/step - loss: 0.1253 - accuracy: 0.9617
Epoch 8/20
103/103 [==============================] - 1s 7ms/step - loss: 0.1192 - accuracy: 0.9685
Epoch 9/20
103/103 [==============================] - 1s 7ms/step - loss: 0.1067 - accuracy: 0.9740
Epoch 10/20
103/103 [==============================] - 1s 7ms/step - loss: 0.1023 - accuracy: 0.9743
Epoch 11/

/usr/local/lib/python3.9/dist-packages/scikeras/wrappers.py:301: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(


Epoch 1/20
103/103 [==============================] - 2s 7ms/step - loss: 0.5024 - accuracy: 0.8757
Epoch 2/20
103/103 [==============================] - 1s 6ms/step - loss: 0.2650 - accuracy: 0.9036
Epoch 3/20
103/103 [==============================] - 1s 7ms/step - loss: 0.2015 - accuracy: 0.9198
Epoch 4/20
103/103 [==============================] - 1s 6ms/step - loss: 0.1694 - accuracy: 0.9357
Epoch 5/20
103/103 [==============================] - 1s 6ms/step - loss: 0.1507 - accuracy: 0.9474
Epoch 6/20
103/103 [==============================] - 1s 7ms/step - loss: 0.1338 - accuracy: 0.9633
Epoch 7/20
103/103 [==============================] - 1s 7ms/step - loss: 0.1236 - accuracy: 0.9672
Epoch 8/20
103/103 [==============================] - 1s 6ms/step - loss: 0.1092 - accuracy: 0.9721
Epoch 9/20
103/103 [==============================] - 1s 6ms/step - loss: 0.1080 - accuracy: 0.9725
Epoch 10/20
103/103 [==============================] - 1s 6ms/step - loss: 0.1051 - accuracy: 0.9767

/usr/local/lib/python3.9/dist-packages/scikeras/wrappers.py:301: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(


307/307 [==============================] - 4s 5ms/step - loss: 0.3198 - accuracy: 0.8784
Epoch 2/20
307/307 [==============================] - 2s 5ms/step - loss: 0.1598 - accuracy: 0.9406
Epoch 3/20
307/307 [==============================] - 2s 5ms/step - loss: 0.1022 - accuracy: 0.9712
Epoch 4/20
307/307 [==============================] - 2s 7ms/step - loss: 0.0830 - accuracy: 0.9782
Epoch 5/20
307/307 [==============================] - 2s 6ms/step - loss: 0.0724 - accuracy: 0.9812
Epoch 6/20
307/307 [==============================] - 2s 5ms/step - loss: 0.0738 - accuracy: 0.9792
Epoch 7/20
307/307 [==============================] - 2s 5ms/step - loss: 0.0632 - accuracy: 0.9835
Epoch 8/20
307/307 [==============================] - 2s 5ms/step - loss: 0.0610 - accuracy: 0.9837
Epoch 9/20
307/307 [==============================] - 2s 5ms/step - loss: 0.0560 - accuracy: 0.9839
Epoch 10/20
307/307 [==============================] - 2s 5ms/step - loss: 0.0558 - accuracy: 0.9843
Epoch 11/2

In [80]:
# Obtener los mejores hiperparámetros y su rendimiento en los datos de entrenamiento
best_params = grid_result.best_params_
best_score = grid_result.best_score_

# Imprimir los mejores hiperparámetros y su rendimiento
print("Mejores hiperparámetros encontrados: ", best_params)
print("Rendimiento del modelo con los mejores hiperparámetros: ", best_score)


Mejores hiperparámetros encontrados:  {'batch_size': 16, 'dropout_rate': 0.1, 'epochs': 20}
Rendimiento del modelo con los mejores hiperparámetros:  0.9869378583244451


In [83]:
batch_size = 16
steps_per_epoch = (X_train.shape[0]//batch_size)
validation_steps = (x_normalised.shape[0] // batch_size)

es = EarlyStopping(monitor='accuracy', mode='min', verbose=1, patience=200)


In [85]:
# Obtener el mejor estimador del resultado de la optimización de hiperparámetros
best_model = grid_result.best_estimator_

# Entrenar el modelo con los mejores hiperparámetros
best_model.fit(X_train, y_train, epochs=best_params['epochs'], batch_size=best_params['batch_size'])


/usr/local/lib/python3.9/dist-packages/scikeras/wrappers.py:301: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(


Epoch 1/20
307/307 [==============================] - 4s 5ms/step - loss: 0.3182 - accuracy: 0.8900
Epoch 2/20
307/307 [==============================] - 2s 5ms/step - loss: 0.1567 - accuracy: 0.9427
Epoch 3/20
307/307 [==============================] - 2s 5ms/step - loss: 0.1052 - accuracy: 0.9733
Epoch 4/20
307/307 [==============================] - 2s 5ms/step - loss: 0.0788 - accuracy: 0.9806
Epoch 5/20
307/307 [==============================] - 2s 5ms/step - loss: 0.0683 - accuracy: 0.9812
Epoch 6/20
307/307 [==============================] - 2s 6ms/step - loss: 0.0642 - accuracy: 0.9824
Epoch 7/20
307/307 [==============================] - 2s 7ms/step - loss: 0.0582 - accuracy: 0.9843
Epoch 8/20
307/307 [==============================] - 2s 5ms/step - loss: 0.0548 - accuracy: 0.9849
Epoch 9/20
307/307 [==============================] - 2s 5ms/step - loss: 0.0513 - accuracy: 0.9851
Epoch 10/20
307/307 [==============================] - 2s 5ms/step - loss: 0.0492 - accuracy: 0.9861

KerasClassifier(
	model=None
	build_fn=<function create_model at 0x7fabde4a18b0>
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=16
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=20
	dropout_rate=0.1
	class_weight=None
)

In [86]:
%%time 

# Entrenar el modelo
model.fit(X_train, y_train,
    steps_per_epoch=steps_per_epoch,
    epochs = 35,
    validation_data=(X_test, y_test),
    validation_steps=validation_steps,
    batch_size = batch_size,
    callbacks=[es],
    verbose=1)

/usr/local/lib/python3.9/dist-packages/scikeras/wrappers.py:301: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(


Epoch 1/35
300/306 [============================>.] - ETA: 0s - loss: 0.3465 - accuracy: 0.8746

306/306 [==============================] - 4s 7ms/step - loss: 0.3440 - accuracy: 0.8756 - val_loss: 0.1845 - val_accuracy: 0.9305
Epoch 2/35
306/306 [==============================] - 2s 6ms/step - loss: 0.1833 - accuracy: 0.9318
Epoch 3/35
306/306 [==============================] - 2s 6ms/step - loss: 0.1200 - accuracy: 0.9621
Epoch 4/35
306/306 [==============================] - 2s 6ms/step - loss: 0.0843 - accuracy: 0.9783
Epoch 5/35
306/306 [==============================] - 2s 6ms/step - loss: 0.0756 - accuracy: 0.9822
Epoch 6/35
306/306 [==============================] - 2s 8ms/step - loss: 0.0642 - accuracy: 0.9818
Epoch 7/35
306/306 [==============================] - 2s 5ms/step - loss: 0.0658 - accuracy: 0.9834
Epoch 8/35
306/306 [==============================] - 2s 5ms/step - loss: 0.0598 - accuracy: 0.9855
Epoch 9/35
306/306 [==============================] - 2s 5ms/step - loss: 0.0541 - accuracy: 0.9851
Epoch 10/35
306/306 [==============================] - 2s 5ms/step - 

KerasClassifier(
	model=None
	build_fn=<function create_model at 0x7fabde4a18b0>
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=None
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=1
	dropout_rate=0.1
	class_weight=None
)

In [90]:
# Obtener el mejor estimador del resultado de la optimización de hiperparámetros
best_model = grid_result.best_estimator_

# Evaluar el modelo en datos de prueba
y_pred = best_model.predict(X_test)
accuracy = best_model.score(X_test, y_test)
print('test_accuracy: %.2f' % (accuracy * 100.0))


132/132 [==============================] - 0s 2ms/step
test_accuracy: 99.05


In [91]:
from sklearn.metrics import classification_report

# Hacer predicciones sobre los datos de prueba
y_pred = model.predict(X_test)

# Convertir las probabilidades en etiquetas binarias (0 o 1) usando un umbral de 0.5
y_pred_binary = (y_pred > 0.5).astype(int)

# calcular métricas de evaluación
from sklearn.metrics import accuracy_score,f1_score, precision_score, recall_score
acc = accuracy_score(y_test, y_pred_binary)*100
f1 = f1_score(y_test, y_pred_binary, average='weighted')*100
prec = precision_score(y_test, y_pred_binary, average='weighted')*100
rec = recall_score(y_test, y_pred_binary, average='weighted')*100

# imprimir las métricas
print("Accuracy:", acc)
print("F1 score:", f1)
print("Precision:", prec)
print("Recall:", rec)

66/66 [==============================] - 0s 3ms/step
Accuracy: 99.0
F1 score: 98.99993537085227
Precision: 99.01608312745928
Recall: 99.0
